# AdaFlix Milestone 2

This notebook presents some preprocessing of our datasets, as well as an initial analysis of factors that contribute to a movie's financial success. In the first part of the notebook we clean the datasets and come up with features that will be used to perform a regression analysis. The analysis will tell us what the significant factors of a movie's financial success are.

# Imports and loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import statsmodels.api as sm
import seaborn as sns
from loads import *

PATH_FOLDER = "MovieSummaries/"

In [ ]:
# Loading character metadata DataFrame
character_df = load_character_metadata()
# Loading movie metadata DataFrame
movie_df = load_movie_metadata()
# Loading plot summaries DataFrame
plot_summaries_df = load_plot_summaries()
# Loading TVTropes DataFrame
tvtropes_df = load_tvtropes()
# Loading name clusters DataFrame
name_clusters_df = load_name_clusters()
# Loading inflation DataFrame
inflation = pd.read_excel("external_dataset/inflation.xlsx", header=11)

# Preprocessing

## CLEANING

#### MOVIE METADATA Cleaning

We are first extracting variables that have the form (ID : variable) tuples to be able to use these features.

In [ ]:
# Extracting and transforming the 'Movie languages (Freebase ID:name tuples)' column
# Applying a lambda function to convert the JSON-like string to a list of language names
movie_df['Movie Languages'] = movie_df['Movie languages (Freebase ID:name tuples)'].apply(lambda x: list(json.loads(x).values()))
# Extracting and transforming the 'Movie countries (Freebase ID:name tuples)' column
# Applying a lambda function to convert the JSON-like string to a list of country names
movie_df['Movie Countries'] = movie_df['Movie countries (Freebase ID:name tuples)'].apply(lambda x: list(json.loads(x).values()))
# Extracting and transforming the 'Movie genres (Freebase ID:name tuples)' column
# Applying a lambda function to convert the JSON-like string to a list of genre names
movie_df['Movie Genres'] = movie_df['Movie genres (Freebase ID:name tuples)'].apply(lambda x: list(json.loads(x).values()))

A lot of columns in the dataframe contain IDs that we don't need, we are thus, dropping them.

In [ ]:
# List of columns to be dropped from the DataFrame
movie_columns_to_drop = ['Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)','Movie genres (Freebase ID:name tuples)','Freebase movie ID']
# Dropping specified columns from the DataFrame
movie_df.drop(movie_columns_to_drop, axis=1, inplace=True)

Our main interest is to see what features result in higher boxe office revenues, so we need to drop movies with missing revenue (Nan).

In [ ]:
# Dropping rows from the DataFrame where 'Movie box office revenue' is NaN
movie_df.dropna(subset=['Movie box office revenue'], inplace=True)

One of our feature of interest is the release date of the movies. We are thus, processing the column, for it to be usable in our analysis. We extract Year and Month from the Movie release date (date format YYYY and YYYY-MM-DD can't be dealt together).

In [ ]:
# Dropping rows where 'Movie release date' is NaN
movie_df.dropna(subset=["Movie release date"], inplace=True)
# Create a Year column in the movie dataframe
movie_df['Year'] = movie_df["Movie release date"].apply(lambda x: str(x)[:4]).astype(int)
# Create a Month column in the movie dataframe
movie_df['Month'] = pd.to_datetime(movie_df['Movie release date'],errors='coerce').dt.month

### Inflation

To be able to compare the the relative box office revenues across different years, we need to take inflation into account.

In [ ]:
# Calculate cumulative inflation to 2022 using the last available annual inflation rate
inflation["Cumulative Inflation to 2022"] = inflation.iloc[-1].Annual / inflation["Annual"]
# Merge the movie DataFrame with the inflation DataFrame on the 'Year' column
movie_with_inflation = pd.merge(movie_df, inflation, on='Year')
# Calculate revenue with inflation by multiplying 'Movie box office revenue' with the 'Cumulative Inflation to 2022'
movie_with_inflation['Revenue with inflation'] = movie_with_inflation['Movie box office revenue'] * movie_with_inflation['Cumulative Inflation to 2022']
movie_with_inflation.drop(['Cumulative Inflation to 2022', 'Annual'], axis=1, inplace=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# Plot for Revenue without inflation
ax1.scatter(movie_with_inflation['Year'], movie_with_inflation['Movie box office revenue'], label='Revenue without inflation')
ax1.set_xlabel('Year')
ax1.set_ylabel('Revenue without inflation')
ax1.legend()

# Plot for Revenue with inflation
ax2.scatter(movie_with_inflation['Year'], movie_with_inflation['Revenue with inflation'], label='Revenue with inflation')
ax2.set_xlabel('Year')
ax2.set_ylabel('Revenue with inflation')
ax2.legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

The original distribution of the inflated movie box office revenue exhibits a heavy-tailed pattern, with a few movies achieving extremely high revenues. By applying a logarithmic transformation on the x-axis, we approximate a "more" normal distribution, making the data more amenable to statistical analyses that assume normality.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 6))

# Plotting the original distribution and the distribution after applying log function on the revenue feature
movie_with_inflation['Revenue with inflation'].hist(ax=axs[0], bins=100, color='blue', edgecolor='black')
movie_with_inflation['log_revenues'] = np.log(movie_with_inflation['Revenue with inflation'])
movie_with_inflation['log_revenues'].hist(ax=axs[1], bins=100, color='blue', edgecolor='black')
axs[0].set_title('Distribution of Box Office Revenue with inflation')
axs[0].set_xlabel('Inflated Box Office Revenue')
axs[0].set_ylabel('Number of movies')
axs[1].set_title('Distribution of log(Box Office Revenue with inflation)')
axs[1].set_xlabel('Log inflated Box Office Revenue')
axs[1].set_ylabel('Number of movies')

#### CHARACTER METADATA Cleaning

Again, a lot of columns in the dataframe contain IDs that we don't need, as well as actors' heights, we are thus, dropping them.

In [ ]:
# Dropping unnecessary character-related columns from the DataFrame
character_columns_to_drop = ['Freebase movie ID', 'Actor ethnicity (Freebase ID)','Freebase character ID','Freebase actor ID','Actor height (in meters)']
character_df.drop(character_columns_to_drop, axis=1, inplace=True)

## Splitting

We are now splitting the dataframes into many small different dataframes for each feature. Each small df has one feature and 'log_revenues'. These small dataframes are created for the analysis to be easier.

In [ ]:
# Extracting 'Movie runtime' and 'log_revenues' columns into a new dataframe
movie_runtime_df = movie_with_inflation[['Movie runtime', 'log_revenues']].copy()
# Extracting 'Movie Languages' and 'log_revenues' columns into a new dataframe
movie_languages_df = movie_with_inflation[['Movie Languages', 'log_revenues']].copy()
# Extracting 'Movie Countries' and 'log_revenues' columns into a new dataframe
movie_countries_df = movie_with_inflation[['Movie Countries', 'log_revenues']].copy()
# Extracting 'Movie Genres' and 'log_revenues' columns into a new dataframe
movie_genres_df = movie_with_inflation[['Movie Genres', 'log_revenues']].copy()
# Extracting 'Month' and 'log_revenues' columns into a new dataframe
months_df = movie_with_inflation[['Month', 'log_revenues']].copy()
# Extracting 'Year' and 'log_revenues' columns into a new dataframe
years_df = movie_with_inflation[['Year', 'log_revenues']].copy()

Drop movies in each small dataframe, only for the missing values (Nan) of the concerned feature

In [ ]:
# Drop NaNs in each small dataframes
movie_runtime_df.dropna(subset=['Movie runtime'], inplace=True)
movie_languages_df.dropna(subset=['Movie Languages'], inplace=True)
movie_countries_df.dropna(subset=['Movie Countries'], inplace=True)
movie_genres_df.dropna(subset=['Movie Genres'], inplace=True)
years_df.dropna(subset=['Year'], inplace=True) #no nan but safer
months_df.dropna(subset=['Month'], inplace=True)

We then need to merge character and movie dataframes, to be able to extract information on actors and their impact on box office revenues

In [ ]:
# Merging character_df and movie_with_inflation DataFrames on 'Wikipedia movie ID'
# Dropping the 'Movie release date' column from character_df to avoid duplicate columns after the merge
character_movie_merged_df = pd.merge(character_df.drop(['Movie release date'],axis=1), movie_with_inflation, on=['Wikipedia movie ID'])


One of our feature will be the percentage of women actresses in movies, so we need to get this information

In [ ]:
# Create a new DataFrame containing relevant columns from the original DataFrame
actor_gender_movie_df = character_movie_merged_df[['Movie name','log_revenues','Actor name','Actor gender']].copy()
# Drop rows with missing values
actor_gender_movie_df.dropna(inplace=True)
# Compute percentage of women in each movies
actor_gender_movie_df['Percentage Women'] = (actor_gender_movie_df.groupby('Movie name')['Actor gender']
                                              .transform(lambda x: (x == 'F').mean() * 100)
                                              .fillna(0)  # Fill NaN with 0
                                              .astype(int)  # Ensure the column is of int type
                                             )

In [ ]:
# Creating a DataFrame with actor gender, actor name, and log_revenues from the merged DataFrame
actor_gender_df = actor_gender_movie_df[['Percentage Women','Actor name','log_revenues']].copy()
# Removing duplicate rows from actor_gender_df based on all columns
actor_gender_df.drop_duplicates()
# Creating a new DataFrame with percentage women and log_revenues, dropping rows with missing percentage women
gender_df = actor_gender_df[['Percentage Women','log_revenues']].copy()
gender_df.dropna(subset=['Percentage Women'], inplace=True)

In [ ]:
# Creating a DataFrame with actor name, actor age at movie release, and log_revenues from the merged DataFrame
actor_df = character_movie_merged_df[['Actor name','Actor age at movie release','log_revenues']].copy()
actor_df.dropna()

## Quantization

In this section we transform runtimes, female percentages and years into categories to be able to run a regression analysis with categorical features. For example, if the female percentage of movie x is 23%, it will be associated to the interval [20%,30%]. Similarly, if the movie runtime of movie x is 123 minutes, it will be associated to the interval [100min,130min].

In [ ]:
# Calculate the minimum and maximum years in the 'Year' column
min_year = movie_with_inflation['Year'].min()
max_year = movie_with_inflation['Year'].max()

# Create intervals of 10 years and assign labels based on the minimum and maximum years
movie_with_inflation['year_intervals'] = pd.cut(movie_with_inflation['Year'],
                                    bins=range(min_year, max_year + 11, 10),  # Define bins for 10-year intervals
                                    labels=[f"{start}-{start + 9}" for start in range(min_year, max_year, 10)],  # Create labels for intervals
                                    include_lowest=True)  # Include the lowest value in the interval

# The 'year_intervals' column now contains the corresponding interval for each year

In [ ]:
# Cast 'Movie runtime' to integers
movie_with_inflation.dropna(subset=['Movie runtime'], inplace=True)
movie_with_inflation['Movie runtime'] = movie_with_inflation['Movie runtime'].astype(int)

# Calculate the minimum and maximum runtimes in the 'Movie runtime' column
min_runtime = movie_with_inflation['Movie runtime'].min()
max_runtime = movie_with_inflation['Movie runtime'].max()

# Create intervals of 30 for runtimes until 220, and a separate interval for values greater than 220 but smaller than the maximum runtime
runtime_bins = [i for i in range(min_runtime, min(max_runtime, 221), 30)] + [float('inf')]

# Create labels for the runtime intervals
runtime_labels = [f"{start}-{start + 29}" if start + 29 < min(max_runtime, 220) else f"{min(max_runtime, 220)+1}-{max_runtime}" for start in range(min_runtime, min(max_runtime, 221), 30)]

# Assign the intervals to the 'Movie runtime' column
movie_with_inflation['runtime_intervals'] = pd.cut(movie_with_inflation['Movie runtime'],
                                       bins=runtime_bins,
                                       labels=runtime_labels,
                                       include_lowest=True)

# The 'runtime_intervals' column now contains the corresponding interval for each runtime

In [ ]:
# Create intervals of size 10 for the 'Percentage Women' column
gender_df['Percentage Women Intervals'] = pd.cut(gender_df['Percentage Women'],
                                                             bins=range(0, 101, 10),
                                                             labels=[f"{start}-{start + 9}" for start in range(0, 100, 10)],
                                                             include_lowest=True)

In [ ]:
def disagregate_list_feature(df, feature_name):
    #Create new df
    result_df = df.copy()
    # Step 1: Get the unique set of languages
    unique_features = set(feature for features in result_df[feature_name] for feature in features)

    # Step 2: Create binary columns for each language
    for feature in unique_features:
        result_df[feature] = result_df[feature_name].apply(lambda x: feature in x)

    # Step 3: Drop the original 'Languages' column
    result_df.drop(feature_name, axis=1, inplace=True)

    # Resulting DataFrame
    return result_df


In [ ]:
def regression_analysis(df, revenue_string):
    X = df.drop([revenue_string], axis=1)
    y = df[revenue_string]

    X = X.astype(int)

    X = sm.add_constant(X)


    model = sm.OLS(y, X).fit()

    print(model.summary())
    return model

def display_regression_result(model):
    # Extract coefficients and corresponding character names
    coefficients = model.params[1:]  # Exclude the intercept
    feature = coefficients.index
    p_values = model.pvalues[1:]
    # Create a DataFrame to store coefficients and character names
    coefficients_df = pd.DataFrame({'Feature': feature, 'Coefficient': coefficients, 'p-value': p_values})
    #print lines of 10 best and worst coefficients
    print('Top 10 more successful features with coefficients and p-values:')
    print(coefficients_df.sort_values(by='Coefficient', ascending=False).head(10))
    print('Top 10 more successful features with coefficients and p-values:')
    print(coefficients_df.sort_values(by='Coefficient', ascending=False).tail(10))


# Data analysis

We know have everything in place to perform our analysis. We will start by running a regression analysis on our features to see what are the characteristics of movies that give best box office revenues.

#### Movie genre

In [ ]:
movie_genres_reg_df = disagregate_list_feature(movie_genres_df, 'Movie Genres')

In [ ]:
model = regression_analysis(movie_genres_reg_df, 'log_revenues')

In [ ]:
display_regression_result(model)

#### Movie countries

In [ ]:
movie_countries_reg_df = disagregate_list_feature(movie_countries_df, 'Movie Countries')

In [ ]:
model = regression_analysis(movie_countries_reg_df, 'log_revenues')

In [ ]:
display_regression_result(model)

#### Movie languages

In [ ]:
movie_languages_reg_df = disagregate_list_feature(movie_languages_df, 'Movie Languages')

In [ ]:
model = regression_analysis(movie_languages_reg_df, 'log_revenues')

In [ ]:
display_regression_result(model)

# Soup regression formula (in progress)

In [ ]:
movie_with_inflation

In [ ]:
formula_df = movie_with_inflation.copy()
formula_df = disagregate_list_feature(formula_df, 'Movie Languages')
formula_df = disagregate_list_feature(formula_df, 'Movie Countries')
formula_df = disagregate_list_feature(formula_df, 'Movie Genres')

In [ ]:
formula_df.drop(['Wikipedia movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Year', 'Movie runtime', 'Revenue with inflation'], axis=1, inplace=True)
formula_df

In [ ]:
formula_df = pd.get_dummies(formula_df, columns=['Month', 'year_intervals', 'runtime_intervals'])
formula_df

In [ ]:
model = regression_analysis(formula_df, 'log_revenues')